In [1]:
import os
os.chdir('../')
%pwd

'd:\\Programming\\MachineLearning\\MLOPs\\End to End ML PipeLine'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
from MLOps.constants import *
from MLOps.utils.common import read_yaml, create_directories

Constants module loaded.


In [ ]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH, schema_file_path=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        create_directories(list_of_directories=[self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        data_ingestion_config = self.config.data_ingestion
        create_directories([data_ingestion_config.root_dir])
        return DataIngestionConfig(
            root_dir=data_ingestion_config.root_dir,
            source_URL=data_ingestion_config.source_URL,
            local_data_file=data_ingestion_config.local_data_file,
            unzip_dir=data_ingestion_config.unzip_dir
        )

In [5]:
import os
import urllib.request as request
import zipfile
from MLOps import logger
from MLOps.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        create_directories([self.config.root_dir])
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            logger.info(f"Downloading file from {self.config.source_URL} to {self.config.local_data_file}")
            request.urlretrieve(self.config.source_URL, self.config.local_data_file)
            logger.info(f"Downloaded {get_size(self.config.local_data_file)} bytes")
        else:
            logger.info(f"File already exists at {self.config.local_data_file}")
    
    def extract_zip_file(self):
        logger.info(f"Extracting zip file to {self.config.unzip_dir}")
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)
        logger.info("Extraction completed")
        

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

artifacts
Creating files
Directory already exists: artifacts
Creating files
Directory already exists: artifacts\data_ingestion
Creating files
Directory already exists: artifacts\data_ingestion
2025-07-02 01:12:49,021 - MLOpsLogger - INFO - File already exists at artifacts/data_ingestion/data.zip
2025-07-02 01:12:49,023 - MLOpsLogger - INFO - Extracting zip file to artifacts/data_ingestion
2025-07-02 01:12:49,036 - MLOpsLogger - INFO - Extraction completed
